# Análise de sentimento por Engenharia de Prompts

- Utilizar o groq.com para usar a API do Llama 3 70B para fazer análise de sentimentos do IMDB. É um enunciado bem livre e vamos acompanhando durante a semana em função dos resultados parciais que vocês conseguem fazer.

# 1. Imports

In [1]:
import torch
import pandas as pd
from datasets import load_dataset
import os
from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

/Users/victorgmoreno/miniconda3/envs/ia024/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Dataset

I am going to use the IMDB dataset and its partitions

In [14]:
# Load the IMDb movie reviews dataset
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')
test_dataset  = test_dataset.shuffle(seed=42).select(range(1000)) # avaliação com 1000 amostras aleatórias do conjunto de teste

# 3. Groq

Groq's API stands out for its speed and efficiency, making it a viable option for developers looking to implement real-time interactions with LLMs in their applications. To use the Groq API, developers need to install the relevant client libraries and set up their API keys.

## 3.1. Basic Groq Test

In [9]:
# Basic Test Groq
client = Groq(api_key=os.environ.get("GROQ_API_KEY"),)

chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "Explain the importance of fast language models",}],
    model="llama3-70b-8192",)

# print(chat_completion.choices[0].message.content)

## 3.2. Groq Llama 3 to Sentiment Aanlysis

In [16]:
def sentiment_analysis(text):
    # Criando o prompt para análise de sentimentos
    prompt = f"Analyze the sentiment of this movie review: '{text}'. Is it positive, negative, or neutral? \
    Your answer should ONLY the tags 'positive', 'negative', 'neutral', nothing else"
    
    # Chamando a API com o prompt
    response = client.chat.completions.create(
        messages=[{"role": "system", "content": prompt}],
        model="llama3-70b-8192"
    )
    
    # Retornando a resposta do modelo
    return response.choices[0].message.content

# Exemplo de uso
review = "The movie was fantastic! The story was engaging and the characters were incredibly well-developed."
print(sentiment_analysis(review))

positive


In [18]:
train_dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be